In [ ]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os 
import sys
import json
import scipy.signal
import scipy.stats
import scipy.interpolate
import pickle as pkl
import scipy.optimize

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')
DATAPATH = os.path.join(PATH, 'datasets/data')


def linear_fit(x, a, b):
    
    return a + b * x



# load data

In [ ]:
os.listdir(os.path.join(RESULTPATH, 'scores'))

In [ ]:
os.listdir(os.path.join(DATAPATH))

In [ ]:
#test_data = mf.data.MFDataset(os.path.join(DATAPATH, '211105_84_25_2cm_random.h5'))
template_data = mf.data.MFDataset(os.path.join(DATAPATH, '211027_84_25_2cm.h5'))

#beamforming_result = np.load(os.path.join(RESULTPATH,'211005_mf_84_100_template_ideal_scores_bottom_tri.npy'))

In [ ]:
template_metadata = pd.DataFrame(template_data.metadata)
#test_metadata = pd.DataFrame(test_data.metadata)

In [ ]:
scores1 = np.load(os.path.join(RESULTPATH, 'scores', '211104_84_25_2cm_template_on_template.npy'))
scores2 = np.load(os.path.join(RESULTPATH, 'scores', '211108_84_25_2cm_template_on_template_1slice.npy'))

# pick signal, sort scores with metadata, normalize

In [ ]:
signal_angle = 89.9
signal_energy = 18587.5
var = 1.38e-23 * 10 * 50 * 200e6
nslice = 8192

signal_index = template_metadata[(template_metadata['energy'] == signal_energy) & (template_metadata['theta_min'] == signal_angle)].index[0]

#signal_index = sorted_inds[355]
#signal_angle = test_metadata.iloc[signal_index]['theta_min']
#signal_energy = test_metadata.iloc[signal_index]['energy']
print(signal_index, signal_angle, signal_energy)

#sns.set_theme(style='ticks', context='talk')
#fig = plt.figure(figsize=(13,13))
#ax = fig.add_subplot(1,1,1)

#img = ax.imshow(scores[signal_index, :].reshape(121, 101), aspect='auto', interpolation='none')
#cbar = fig.colorbar(img)

signal = template_data.data[signal_index, :]
sample_inds = np.arange(0, 60 * 3 * nslice, 1).reshape((60, 3 * nslice))[:, 0:nslice].flatten()

#signal = test_data.data[signal_index, :]

norm1 = abs(1 / np.sqrt(var * np.vdot(signal, signal)))
norm2 = abs(1 / np.sqrt(var * np.vdot(signal[sample_inds], signal[sample_inds])))

energy_array = template_metadata['energy'].array
angle_array = template_metadata['theta_min'].array

#test_energies = np.array(test_metadata['energy'].array).squeeze()
#test_angles = np.array(test_metadata['theta_min'].array).squeeze()

unique_energy = np.sort(np.unique(template_metadata['energy'].array))
unique_angle = np.sort(np.unique(template_metadata['theta_min'].array))

energy_grid, angle_grid = np.meshgrid(unique_energy, unique_angle)
sorting_indexes = []

for i, pair in enumerate(zip(energy_grid.flatten(), angle_grid.flatten())):
    
    sorting_indexes.append(template_metadata[(template_metadata['energy'] == pair[0]) & (template_metadata['theta_min'] == pair[1])].index[0])
    
sorting_indexes = np.array(sorting_indexes)

#print(sorting_indexes.shape)

sorted_scores1 = (scores1[signal_index, sorting_indexes] * norm1).reshape((energy_grid.shape))
sorted_scores2 = (scores2[signal_index, sorting_indexes] * norm2).reshape((energy_grid.shape))

#norm_scores1 = sorted_scores1 / (np.max(sorted_scores1) - np.min(sorted_scores1))
#norm_scores2 = sorted_scores2 / (np.max(sorted_scores2) - np.min(sorted_scores2))

# plot

In [ ]:
sns.set_theme(style='ticks', context='talk')
cmap = sns.color_palette('vlag', as_cmap=True)
fig = plt.figure(figsize=(13,8))

from matplotlib.colors import LogNorm

shift = abs(np.max(sorted_scores1-sorted_scores2) - abs(np.min(sorted_scores1-sorted_scores2))) / 2

ax = fig.add_subplot(1,1,1)
img = ax.imshow((sorted_scores1-sorted_scores2), aspect='auto', interpolation='none', cmap=cmap, 
                norm=None, extent=(18575, 18600, 90, 84), vmin=-np.max(sorted_scores1-sorted_scores2),
               vmax=np.max(sorted_scores1-sorted_scores2))
cbar = fig.colorbar(img)

ax.set_xlabel('Energy (eV)')
ax.set_ylabel('Pitch Angle (deg)')
name = f'211108_offaxis2cm_3timeslice_minus_1timeslice_pa{signal_angle}_energy{signal_energy}.png'
#ax.plot(signal_energy, signal_angle, 'r.')

plt.tight_layout()

plt.savefig(os.path.join(PLOTPATH, name))